In [15]:
import pandas as pd
import requests
import numpy as np
import json
from datetime import datetime
from gvol import GVol
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Insert your GVol API KEY
GVOL_API_KEY = "GVOL_API_KEY"
gvol_client = GVol(header='gvol-lite',gvol_api_key=GVOL_API_KEY)
url = "https://app.pinkswantrading.com/graphql"
pd.set_option('display.max_columns', None)

### Query BTC Variance Premium data from Amberdata Derivaitves

In [16]:
symbol = "BTC"

payload = '{{"query":"query DVolVariancePremium($symbol: SymbolEnumType) {{\\n\\t\\tdvolVariancePremium(symbol: $symbol) {{\\n\\t\\t\\tdvolImpliedRvDate\\n\\t\\t\\tinstrument\\n\\t\\t\\tdvolOpen30Days\\n\\t\\t\\tparkinsonHv\\n\\t\\t\\tvariancePremium\\n\\t\\t}}\\n\\t}}","variables":{{"symbol":"{}"}}}}'.format(symbol)
headers = {
  'x-oracle': GVOL_API_KEY,
  'Content-Type': 'application/json',
  'accept': '*/*',
  'Accept-Language': 'en-US,en;q=0.9'
}

response = requests.request("GET", url, headers=headers, data=payload)

# Parse the response text into JSON format
data = json.loads(response.text)

# Convert the JSON data into a pandas DataFrame
# The json_normalize function is used to flatten the JSON into a table
btc_iv_rv = pd.json_normalize(data['data']['dvolVariancePremium'])

# Convert 'date' column to datetime and set index
btc_iv_rv['dvolImpliedRvDate'] = pd.to_datetime(btc_iv_rv['dvolImpliedRvDate'], unit='ms')
btc_iv_rv.set_index('dvolImpliedRvDate', inplace=True)
btc_iv_rv = btc_iv_rv.dropna(axis=0)

btc_iv_rv.head()

,instrument,dvolOpen30Days,parkinsonHv,variancePremium
dvolImpliedRvDate,,,,
2023-07-11,BTC,43.37,40.75,2.62
2023-07-10,BTC,40.80,40.99,-0.19
2023-07-09,BTC,41.35,41.23,0.12
2023-07-08,BTC,45.63,41.33,4.30
2023-07-07,BTC,43.64,41.52,2.12


### Query ETH Variance Premium data from Amberdata Derivaitves

In [17]:
symbol = "ETH"

payload = '{{"query":"query DVolVariancePremium($symbol: SymbolEnumType) {{\\n\\t\\tdvolVariancePremium(symbol: $symbol) {{\\n\\t\\t\\tdvolImpliedRvDate\\n\\t\\t\\tinstrument\\n\\t\\t\\tdvolOpen30Days\\n\\t\\t\\tparkinsonHv\\n\\t\\t\\tvariancePremium\\n\\t\\t}}\\n\\t}}","variables":{{"symbol":"{}"}}}}'.format(symbol)
headers = {
  'x-oracle': GVOL_API_KEY,
  'Content-Type': 'application/json',
  'accept': '*/*',
  'Accept-Language': 'en-US,en;q=0.9'
}

response = requests.request("GET", url, headers=headers, data=payload)

# Parse the response text into JSON format
data = json.loads(response.text)

# Convert the JSON data into a pandas DataFrame
# The json_normalize function is used to flatten the JSON into a table
eth_iv_rv = pd.json_normalize(data['data']['dvolVariancePremium'])

# Convert 'date' column to datetime and set index
eth_iv_rv['dvolImpliedRvDate'] = pd.to_datetime(eth_iv_rv['dvolImpliedRvDate'], unit='ms')
eth_iv_rv.set_index('dvolImpliedRvDate', inplace=True)
eth_iv_rv = eth_iv_rv.dropna(axis=0)

eth_iv_rv.head()

,instrument,dvolOpen30Days,parkinsonHv,variancePremium
dvolImpliedRvDate,,,,
2023-07-11,ETH,45.56,41.19,4.37
2023-07-10,ETH,40.86,41.44,-0.57
2023-07-09,ETH,40.94,43.55,-2.61
2023-07-08,ETH,41.64,43.59,-1.95
2023-07-07,ETH,42.53,43.61,-1.08


### Prepare Variance Premium Plots

In [18]:
# Define Amberdata color theme
AD_COLORS_1 = ['#1c3664', '#89bed8', '#f16623']

def plot_variance_premium(iv_rv):
    
    # Extract symbol from dataframe
    symbol = iv_rv['instrument'].iloc[0]
    # Create subplots: use two rows and one column
    fig = make_subplots(rows=2, cols=1)

    # Line plot of DVOL
    fig.add_trace(go.Scatter(
        x=iv_rv.index,
        y=iv_rv['dvolOpen30Days'],
        mode='lines',
        name='dvolOpen30Days',
        line=dict(color=AD_COLORS_1[2])  # Set line color to black
    ), row=1, col=1)
    
    # Line plot of Parkinson HV
    fig.add_trace(go.Scatter(
        x=iv_rv.index,
        y=iv_rv['parkinsonHv'],
        mode='lines',
        name='parkinsonHv',
        line=dict(color=AD_COLORS_1[1])  # Set line color
    ), row=1, col=1)

    # Bar plot of Variance Premium
    fig.add_trace(go.Bar(
        x=iv_rv.index,
        y=iv_rv['variancePremium'],
        name='Variance Premium',
        marker_color="black",  # Set marker color
        showlegend=False
    ), row=2, col=1)

    fig.update_layout(
        title=f"{symbol} Variance Premium",
        yaxis_title="Price",
        yaxis2_title="Variance Premium",
        template='plotly_white',
        height=600,
        showlegend=True
    )

    fig.show()

### Call the function to plot BTC DVOL, Parkinson Vol, and Variance Premium

In [19]:
plot_variance_premium(btc_iv_rv)

### Call the function to plot ETH DVOL, Parkinson Vol, and Variance Premium

In [20]:
plot_variance_premium(eth_iv_rv)

### Identify basic statisics for plot BTC DVOL, Parkinson Vol, and Variance Premium

In [21]:
btc_var_desc = btc_iv_rv.describe(percentiles=[.05, .25, .5, .75, .95])
btc_var_desc = btc_var_desc[1:]

btc_var_desc

,dvolOpen30Days,parkinsonHv,variancePremium
mean,76.113753,64.210494,11.903136
std,17.292703,22.540779,21.175715
min,39.750000,24.470000,-70.200000
5%,49.976000,35.696000,-19.979500
25%,62.230000,47.532500,1.312500
50%,76.575000,62.740000,13.805000
75%,88.175000,73.467500,26.355000
95%,103.013500,98.801500,37.842000
max,156.200000,150.450000,75.880000


### Identify basic statisics for plot ETH DVOL, Parkinson Vol, and Variance Premium

In [22]:
eth_var_desc = eth_iv_rv.describe(percentiles=[.05, .25, .5, .75, .95])
eth_var_desc = eth_var_desc[1:]

eth_var_desc

,dvolOpen30Days,parkinsonHv,variancePremium
mean,91.552111,82.682370,8.869716
std,23.701271,34.534475,31.084018
min,40.350000,35.090000,-125.250000
5%,54.299500,41.434500,-38.764000
25%,71.845000,56.392500,-2.707500
50%,93.360000,82.230000,15.275000
75%,107.287500,99.012500,29.025000
95%,125.160500,129.332500,42.122000
max,193.370000,218.940000,96.030000


### Prepare Statistics Plots

In [23]:
def plot_statistics(df, symbol):
    
    # Extract symbol from dataframe
    symbol = symbol

    labels = df.index.tolist()
    dvolOpen30Days = df['dvolOpen30Days'].tolist()
    parkinsonHv = df['parkinsonHv'].tolist()
    variancePremium = df['variancePremium'].tolist()

    fig = go.Figure(data=[
        go.Bar(name='dvolOpen30Days', x=labels, y=dvolOpen30Days, marker_color=AD_COLORS_1[0]),
        go.Bar(name='parkinsonHv', x=labels, y=parkinsonHv, marker_color=AD_COLORS_1[1]),
        go.Bar(name='variancePremium', x=labels, y=variancePremium, marker_color=AD_COLORS_1[2])
    ])

    fig.update_layout(
        barmode='group',
        title=f"{symbol} DVOL, HV, and Variance Premium Descriptive Statistics"
    )
    fig.show()

### Call the function to plot BTC DVOL, Parkinson Vol, and Variance Premium Statistics

In [24]:
plot_statistics(btc_var_desc, "BTC")

### Call the function to plot ETH DVOL, Parkinson Vol, and Variance Premium Statistics

In [25]:
plot_statistics(eth_var_desc, "ETH")

### Query ETH/BTC Variance Premium data from Amberdata Derivaitves

In [26]:
symbol = "ETH"
start_date = "2018-06-30"
end_date = datetime.now().strftime("%Y-%m-%d") # Today's date

payload = '{{"query":"query RealizedVolCorr($symbol:SymbolEnumType, $dateStart:String, $dateEnd:String){{\\n  RealizedVolCorr(symbol:$symbol, dateStart: $dateStart, dateEnd: $dateEnd ) {{\\n    date\\n    currency\\n    corr10Btc\\n    corr30Btc\\n    corr90Btc\\n    corr10Eth\\n    corr30Eth\\n    corr90Eth\\n  }}\\n}}\\n","variables":{{"symbol":"{}","dateEnd":"{}","dateStart":"{}"}}}}'.format(symbol, end_date, start_date)
headers = {
  'x-oracle': GVOL_API_KEY,
  'Content-Type': 'application/json',
  'accept': '*/*',
  'Accept-Language': 'en-US,en;q=0.9'
}

response = requests.request("GET", url, headers=headers, data=payload)

# Parse the response text into JSON format
data = json.loads(response.text)

# Convert the JSON data into a pandas DataFrame
# The json_normalize function is used to flatten the JSON into a table
eth_corr = pd.json_normalize(data['data']['RealizedVolCorr'])

# Convert 'date' column to datetime and set as index
eth_corr['date'] = pd.to_datetime(eth_corr['date'], unit='ms')
eth_corr.set_index('date', inplace=True)

eth_corr = eth_corr.drop(columns=['corr10Eth', 'corr30Eth', 'corr90Eth'])
eth_corr.head()

,currency,corr10Btc,corr30Btc,corr90Btc
date,,,,
2023-07-10,ETH,0.8963,0.8423,0.8291
2023-07-09,ETH,0.6314,0.8517,0.8199
2023-07-08,ETH,0.6494,0.8536,0.8207
2023-07-07,ETH,0.7320,0.8531,0.8202
2023-07-06,ETH,0.7432,0.8567,0.8189


### Prepare ETH/BTC Correlation Plot

In [28]:
AD_COLORS_2 = ['#4c61ab', '#e00d8e', '#89bed8', '#882c91', '#f16623']

def plot_corr(df, column):
    
    # Create the figure
    fig = px.line(df[column], title="ETH/BTC Historical Correlation", color_discrete_sequence=AD_COLORS_2)

    # Update axes
    fig.update_xaxes(title='')
    fig.update_yaxes(title=f"{column}")

    # Hide legend
    fig.update_layout(showlegend=False)

    # Show the figure
    fig.show()

### Call the function to plot ETH/BTC Correlation

In [29]:
plot_corr(eth_corr, 'corr30Btc')